# Airline AI Assistant

This is a simple chatbot that will use the existing functions defined in this notebook to look up flight information and return the prices.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyCU


In [4]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

Here, I will add two tools that include two functions.

First function is to simply return the flight ticket prices based on destination.

Second function, more advanced function, returns the flight ticket information based on the destination and time.

In [8]:
ticket_prices = {"london": "$799", "paris": "$899", "singapore": "$3500", "new york": "$4000"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [9]:
get_ticket_price("singapore")

Tool get_ticket_price called for singapore


'$3500'

In [10]:
# Define time slots
TIME_SLOTS = {
    "morning": "06:00-12:00",
    "afternoon": "12:00-18:00",
    "evening": "18:00-24:00",
    "night": "00:00-06:00"
}

# Define base prices and time-based multipliers
ticket_prices = {
    "london": {
        "base_price": 799,
        "time_multipliers": {
            "morning": 1.2,    # 20% more expensive
            "afternoon": 1.0,  # standard price
            "evening": 0.8,    # 20% cheaper
            "night": 0.6      # 40% cheaper
        }
    },
    "paris": {
        "base_price": 899,
        "time_multipliers": {
            "morning": 1.1,
            "afternoon": 1.0,
            "evening": 0.9,
            "night": 0.7
        }
    },
    "singapore": {
        "base_price": 3500,
        "time_multipliers": {
            "morning": 1.1,
            "afternoon": 1.0,
            "evening": 0.9,
            "night": 0.7
        }
    },
    "new york": {
        "base_price": 4000,
        "time_multipliers": {
            "morning": 1.1,
            "afternoon": 1.0,
            "evening": 0.9,
            "night": 0.7
        }
    }
}

In [11]:
# function with time and destinations

def get_ticket_price_with_time(destination_city, time_slot):
    """
    Get the price of a ticket based on destination and time slot.
    
    Args:
        destination_city (str): The destination city
        time_slot (str): One of 'morning', 'afternoon', 'evening', 'night'
    
    Returns:
        float: The calculated price
    """
    print(f"Tool get_ticket_price_with_time called for {destination_city} at {time_slot}")
    city = destination_city.lower()
    time_slot = time_slot.lower()
    
    if city not in ticket_prices:
        return "Unknown destination"
    
    if time_slot not in TIME_SLOTS:
        return "Invalid time slot"
    
    base_price = ticket_prices[city]["base_price"]
    multiplier = ticket_prices[city]["time_multipliers"][time_slot]
    
    final_price = base_price * multiplier
    return f"${final_price:.2f}"

In [12]:
get_ticket_price_with_time("singapore", "night")

Tool get_ticket_price_with_time called for singapore at night


'$2450.00'

In [14]:
# description is important for LLM to understand the price function

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [15]:
# updated function for LLM
time_price_function = {
    "name": "get_ticket_price_with_time",
    "description": "Get the price of a return ticket to the destination city based on the time of flight. Call this whenever you need to know the ticket price for a specific time, for example when a customer asks 'How much is a morning flight to this city' or 'What are the prices for different times of day'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "time_slot": {
                "type": "string",
                "description": "The time slot for the flight (morning, afternoon, evening, or night)",
                "enum": ["morning", "afternoon", "evening", "night"]
            }
        },
        "required": ["destination_city", "time_slot"],
        "additionalProperties": False
    }
}

In [16]:
#tools

tools = [{"type": "function", "function": price_function}, {"type": "function", "function": time_price_function}]

In [17]:
# updated handle tool call
def handle_tool_call(message):
    responses = []
    
    # Loop through all tool calls
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        
        if tool_call.function.name == "get_ticket_price_with_time":
            city = arguments.get('destination_city')
            time_slot = arguments.get('time_slot')
            price = get_ticket_price_with_time(city, time_slot)
            response = {
                "role": "tool",
                "content": json.dumps({
                    "destination_city": city,
                    "time_slot": time_slot,
                    "price": price
                }),
                "tool_call_id": tool_call.id
            }
            responses.append(response)
        
        elif tool_call.function.name == "get_ticket_price":
            city = arguments.get('destination_city')
            price = get_ticket_price(city)
            response = {
                "role": "tool",
                "content": json.dumps({
                    "destination_city": city,
                    "price": price
                }),
                "tool_call_id": tool_call.id
            }
            responses.append(response)
    
    # Return all responses
    return responses, None  # The second return value is not used in this case

In [18]:
# updated chat function
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses, _ = handle_tool_call(message)
        
        # Add the assistant's message with tool calls
        messages.append(message)
        
        # Add all tool responses
        for response in responses:
            messages.append(response)
            
        # Make a second API call to get the final response
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [19]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Singapore
Tool get_ticket_price_with_time called for Singapore at morning
Tool get_ticket_price_with_time called for Singapore at afternoon
Tool get_ticket_price_with_time called for Singapore at evening
Tool get_ticket_price_with_time called for Singapore at night


# Running this on local LLM

In [21]:
!pip install ollama

  Using cached ollama-0.4.7-py3-none-any.whl.metadata (4.7 kB)
Using cached ollama-0.4.7-py3-none-any.whl (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.20.1
    Uninstalling pydantic_core-2.20.1:
      Successfully uninstalled pydantic_core-2.20.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.8.2
    Uninstalling pydantic-2.8.2:
      Successfully uninstalled pydantic-2.8.2


In [22]:
import ollama

In [25]:
#!ollama pull gemma3:1b

In [26]:
# format the message for Ollama

def format_messages_for_ollama(messages):
    """Convert OpenAI message format to Ollama format"""
    formatted_messages = []
    
    for msg in messages:
        if msg["role"] == "system":
            # Ollama treats system message as a special case
            continue
        elif msg["role"] == "tool":
            # Convert tool responses to assistant messages for Ollama
            formatted_messages.append({
                "role": "assistant",
                "content": f"Tool response: {msg['content']}"
            })
        else:
            # User and assistant messages can be passed as is
            formatted_messages.append(msg)
    
    return formatted_messages

In [29]:
def chat(message, history, use_ollama=False):
    # Convert history from Gradio format to OpenAI format
    formatted_history = []
    for user_msg, assistant_msg in history:
        formatted_history.append({"role": "user", "content": user_msg})
        if assistant_msg:  # Only add assistant message if it exists
            formatted_history.append({"role": "assistant", "content": assistant_msg})
    
    messages = [{"role": "system", "content": system_message}] + formatted_history + [{"role": "user", "content": message}]
    
    if use_ollama:
        # Ollama implementation...
        # (rest of the Ollama code remains the same)
        pass
    else:
        # OpenAI implementation
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

        if response.choices[0].finish_reason == "tool_calls":
            message = response.choices[0].message
            responses, _ = handle_tool_call(message)
            
            messages.append(message)
            for response in responses:
                messages.append(response)
                
            response = openai.chat.completions.create(model=MODEL, messages=messages)
        
        return response.choices[0].message.content

In [30]:
# updated gradio interface to switch between OpenAI and Ollama
with gr.Blocks() as demo:
    gr.Markdown("# FlightAI Assistant")
    
    with gr.Row():
        use_ollama = gr.Checkbox(label="Use Ollama (Local LLM)", value=False)
    
    chatbot = gr.ChatInterface(
        fn=lambda message, history: chat(message, history, use_ollama.value)
    )

demo.launch()

/opt/anaconda3/lib/python3.12/site-packages/gradio/chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price_with_time called for Singapore at morning
Tool get_ticket_price_with_time called for Singapore at afternoon
Tool get_ticket_price_with_time called for Singapore at evening
Tool get_ticket_price_with_time called for Singapore at night
Tool get_ticket_price called for Singapore
